# AlexNet Implementation (ImageNet 2012)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchvision import transforms, datasets

In [4]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=256 * 6 * 6, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=1000)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)  # overlapping pooling
        
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        
        x = x.view(x.size(0), -1)  # 4차원을 1차원으로 펼쳐주는 층 (역할) -> flatten
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        
        x = F.log_softmax(self.fc3(x), dim=1)
        
        return x
    
    @staticmethod
    def transform():
        return transforms.Compose([
            transforms.Resize(226),
            transforms.CenterCrop(227),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # https://pytorch.org/hub/pytorch_vision_alexnet/ 참고
        ])
    
if __name__ == "__main__":
    # if gpu is to be used
    use_cuda = torch.cuda.is_available()
    print("use cuda : ", use_cuda)
    
    FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
    device = torch.device("cuda:0" if use_cuda else "cpu")
    
    net = AlexNet().to(device)
    
    X = torch.randn(size=(1, 3, 227, 227)).type(FloatTensor)
    print(net(X))
    print(summary(net, (3, 227, 227)))

use cuda :  True
tensor([[-6.9131, -6.9201, -6.9216, -6.9221, -6.9077, -6.9061, -6.8979, -6.9210,
         -6.9056, -6.8942, -6.8884, -6.8962, -6.9188, -6.9020, -6.8968, -6.9169,
         -6.9050, -6.8948, -6.9223, -6.9104, -6.9069, -6.9239, -6.9002, -6.8851,
         -6.8931, -6.8950, -6.9114, -6.9318, -6.8813, -6.9597, -6.8877, -6.9042,
         -6.9137, -6.9128, -6.9147, -6.9005, -6.9068, -6.9067, -6.9287, -6.8880,
         -6.8836, -6.9180, -6.9159, -6.8923, -6.9405, -6.8945, -6.9041, -6.9056,
         -6.9333, -6.8934, -6.8956, -6.9207, -6.9295, -6.9062, -6.9297, -6.8925,
         -6.9143, -6.9270, -6.9212, -6.8827, -6.9006, -6.8931, -6.9194, -6.8969,
         -6.9309, -6.9368, -6.8924, -6.9198, -6.8865, -6.8909, -6.9409, -6.9006,
         -6.9169, -6.9218, -6.9039, -6.9240, -6.9374, -6.9132, -6.9064, -6.9224,
         -6.9015, -6.8824, -6.9396, -6.8973, -6.9155, -6.9067, -6.9307, -6.9082,
         -6.8908, -6.9069, -6.9077, -6.9250, -6.8991, -6.8918, -6.9031, -6.9113,
         -6

In [10]:
if __name__ == "__main__":
    # hyper parameter
    batch_size = 32
    num_epochs = 10
    learning_rate = 0.0001
    
    # data load
    root = './ImageNet2012'
    transform = AlexNet.transform()
    train_set = datasets.ImageNet(root=root, split='train', transform=transform, download=True)
    
    test_set = datasets.ImageNt(root=root, split='val', trainsform=transform, download=True)

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in ./ImageNet2012.

In [12]:
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train_t3.tar

--2022-07-28 04:41:15--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train_t3.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 762460160 (727M) [application/x-tar]
Saving to: ‘ILSVRC2012_img_train_t3.tar’

ILSVRC2012_img_trai 100%[===================>] 727.14M   684KB/s    in 4m 42s  

2022-07-28 04:45:58 (2.57 MB/s) - ‘ILSVRC2012_img_train_t3.tar’ saved [762460160/762460160]



In [13]:
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train.tar

--2022-07-28 04:45:59--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147897477120 (138G) [application/x-tar]
Saving to: ‘ILSVRC2012_img_train.tar’

ILSVRC2012_img_trai 100%[===================>] 137.74G  11.2MB/s    in 16h 41m 

2022-07-28 21:27:33 (2.35 MB/s) - ‘ILSVRC2012_img_train.tar’ saved [147897477120/147897477120]



In [14]:
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar

--2022-07-28 21:27:33--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘ILSVRC2012_img_val.tar’

ILSVRC2012_img_val. 100%[===================>]   6.28G   472KB/s    in 1h 41m  

2022-07-28 23:09:05 (1.06 MB/s) - ‘ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]



In [15]:
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_test_v10102019.tar

--2022-07-28 23:09:05--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_test_v10102019.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13685780480 (13G) [application/x-tar]
Saving to: ‘ILSVRC2012_img_test_v10102019.tar’

ILSVRC2012_img_test 100%[===================>]  12.75G  8.02MB/s    in 3h 27m  

2022-07-29 02:37:03 (1.05 MB/s) - ‘ILSVRC2012_img_test_v10102019.tar’ saved [13685780480/13685780480]

